<a href="https://colab.research.google.com/github/maciekdabrowski/PortfolioProjects/blob/main/rtbhouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

In [2]:
from google.colab import files
uploaded = files.upload()

Saving rtbhouse.db to rtbhouse.db


In [4]:
%load_ext sql

In [6]:
%sql sqlite:///rtbhouse.db

In [18]:
import pandas as pd

In [21]:
conn = sqlite3.connect("rtbhouse.db")

In [49]:
query = '''

WITH distinct_products AS (
    SELECT DISTINCT
        id
        , brand
    FROM products
)

, clicks_count AS (
    SELECT
        SUBSTR(c.date, 1, 7) AS month
        , c.campaign_id
        , dp.brand
        , COUNT(c.user_id) AS number_of_clicks
    FROM clicks c
    LEFT JOIN distinct_products dp
        ON c.product_id = dp.id
    GROUP BY month, c.campaign_id, dp.brand
)

, earliest_clicks AS (
    SELECT
        user_id
        , campaign_id
        , product_id
        , dp.brand
        , MIN(date) AS first_click_date
    FROM clicks c
    LEFT JOIN distinct_products dp
        ON c.product_id = dp.id
    GROUP BY
        user_id, campaign_id, product_id, dp.brand
)

, transactions_cte_rn AS (
    SELECT
      transaction_id
      , user_id
      , campaign_id
      , date
      , transaction_value
      , ROW_NUMBER() OVER (PARTITION BY transaction_id, user_id, campaign_id ORDER BY date DESC) AS rn
    FROM transactions
)

, transactions_cte AS (
    SELECT
      transaction_id
      , user_id
      , campaign_id
      , MIN(date) AS earliest_transaction
      , MAX(CASE WHEN rn = 1 THEN transaction_value END) AS latest_value
    FROM transactions_cte_rn
    GROUP BY transaction_id, user_id, campaign_id
)

, post_click_transactions AS (
    SELECT
        SUBSTR(date(t.earliest_transaction), 1, 7) AS month
        , t.campaign_id
        , dp.brand
        , COUNT(DISTINCT t.transaction_id) AS number_of_transactions
        , SUM(t.latest_value) AS value_of_products
    FROM transactions_cte t
    LEFT JOIN transactions_products p
        USING(transaction_id)
    LEFT JOIN distinct_products dp
        ON p.product_id = dp.id
    INNER JOIN earliest_clicks ec
        USING(user_id, campaign_id, brand)
    WHERE t.earliest_transaction > ec.first_click_date
    GROUP BY month, t.campaign_id, dp.brand
)

SELECT
    cc.month
    , cc.campaign_id
    , cc.brand
    , cc.number_of_clicks
    , COALESCE(pct.number_of_transactions, 0) AS number_of_transactions
    , ROUND((0.01 * COALESCE(cc.number_of_clicks,0) + 0.1 * COALESCE(pct.value_of_products,0)), 2) AS advertising_campaign_cost
    , ROUND(COALESCE(pct.value_of_products, 0), 2) AS value_of_products
FROM clicks_count cc
LEFT JOIN post_click_transactions pct
    USING (month, campaign_id, brand)

'''

In [50]:
df_report = pd.read_sql_query(query, conn)

In [51]:
df_report.to_csv('rtbhouse_results.csv', index=False)